# Digit Recognition with CNN + Streamlit Image Upload

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_model():
    model = models.Sequential([
        layers.Reshape((28, 28, 1), input_shape=(28, 28)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [2]:
# Train the CNN
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

model = build_model()
model.fit(x_train, y_train, epochs=3, validation_split=0.1)

model.save("mnist_digit_model.h5")
print("Model Saved!")

Epoch 1/3
1688/1688 [==============================] - 27s 8ms/step - loss: 0.1373 - accuracy: 0.9587 - val_loss: 0.0520 - val_accuracy: 0.9855
Epoch 2/3
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0453 - accuracy: 0.9864 - val_loss: 0.0435 - val_accuracy: 0.9858
Epoch 3/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0303 - accuracy: 0.9904 - val_loss: 0.0442 - val_accuracy: 0.9897
Model Saved!


## Streamlit App for Image Upload & Prediction

In [3]:
%%writefile streamlit_app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import cv2

st.title("Digit Recognition App 🧠🔢")

uploaded_file = st.file_uploader("Upload a 28x28 digit image", type=["png", "jpg", "jpeg"])

if uploaded_file:
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    st.image(img, caption="Uploaded Image", width=150)

    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    model = tf.keras.models.load_model("mnist_digit_model.h5")
    prediction = model.predict(img)
    digit = np.argmax(prediction)

    st.subheader(f"Predicted Digit: {digit}")

Writing streamlit_app.py
